## How to explain a Text Classifier?

- Main objective: to try to understand how the model assigns a text to a label
- Can give insights on how to improve the classification by using other algorithms, adding features,...
- Legal requirement: to prove the algorithm is not biased or doesn't discriminate
- To increase transparancy and therefore confidence in the prediction/classification of the algorithm

### Methods used for explaining the results of an algorithm

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('eclipse_jdt.csv')
df=df[['Title','Description','Priority']]
df=df.dropna()
df['text']= df['Title']+''+df['Description']
df=df.drop(columns=['Title','Description'])

In [8]:
import textacy
import textacy.preprocessing as tprep

preproc = tprep.make_pipeline(
    tprep.replace.urls,
    tprep.remove.html_tags,
    tprep.normalize.hyphenated_words,
    tprep.normalize.quotation_marks,
    tprep.normalize.unicode,
    tprep.remove.accents,
    tprep.remove.punctuation,
    tprep.normalize.whitespace,
    tprep.replace.numbers
)

In [9]:
df['text']=df['text'].apply(preproc)
df=df[df['text'].str.len()>50]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],df['Priority'], test_size=0.2,random_state=42,stratify=df['Priority'])

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=5,ngram_range=(1,2),stop_words='english')
X_train_tf=tfidf.fit_transform(X_train)

In [12]:
X_test_tf=tfidf.transform(X_test)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer(min_df=5,ngram_range=(1,2),stop_words='english')
X_train_counts=count_vect.fit_transform(X_train)
X_test_counts=count_vect.transform(X_test)

In [14]:
count_vect.get_feature_names_out()

array(['0000a000', '0000a000 _number_', '0000c000', ..., 'zzz',
       'zzz _number_', 'zzz public'], dtype=object)

### SVC Models

In [15]:
from sklearn.svm import LinearSVC
model1 = LinearSVC(random_state=0,tol=1e-5)
model1.fit(X_train_tf,Y_train)

LinearSVC(random_state=0, tol=1e-05)

In [16]:
Y_pred = model1.predict(X_test_tf)

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy score', accuracy_score(Y_test, Y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_pred,zero_division=0.0))

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(Y_test,Y_pred,zero_division=0))

In [ ]:
result = pd.DataFrame({'text': X_test.values,'actual':Y_test.values,'Predicted':Y_pred})

result[result['actual']!=result['Predicted']].sample(10)

In [ ]:
print(result.iloc[2611]['text'])

In [ ]:
model1.coef_.shape

In [ ]:
vocabulary=tfidf.vocabulary_
new_vocabulary={value:key for (key,value) in vocabulary.items()}

df_voca=pd.DataFrame(new_vocabulary.items(),columns=['number','feature']).set_index('number')
df_voca.head()

In [ ]:
print(df_voca.loc[20539,'feature'])

In [ ]:
coef= model1.coef_[2]
vocabulary_positions = model1.coef_[2].argsort() # return the indices that would sort an array in ascending order

In [ ]:
top_words=20
top_positive_coef=vocabulary_positions[-top_words:].tolist()
top_negative_coef=vocabulary_positions[:top_words].tolist()

In [ ]:
top_positive_coef

In [ ]:
for i in top_negative_coef:
    print(coef[i])

In [ ]:
for i in top_positive_coef:
    print(df.loc[i,'feature'])

In [ ]:
priop1=pd.DataFrame([[df.loc[c,'feature'], coef[c]] for c in top_positive_coef+top_negative_coef],columns=['feature',"coefficient"]).sort_values("coefficient")

In [ ]:
priop1

In [ ]:
priop1.set_index("feature").plot.barh()

## Using ELI5 to explain Classification results
https://eli5.readthedocs.io/en/latest/index.html

In [17]:
import eli5

count_vect.get_feature_names_out()

eli5.show_weights(model1,vec=count_vect,feature_names=count_vect.get_feature_names_out())

In [28]:
eli5.show_prediction(model1,X_test.iloc[1],vec=count_vect,feature_names=count_vect.get_feature_names_out())

### Using LIME

In [ ]:
from sklearn.svm import SVC
model0 = SVC(kernel="linear",C=1,probability=True, random_state=0,tol=1e-5)
model0.fit(X_train_tf,Y_train)

In [ ]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(tfidf,model0)

In [ ]:
import joblib
joblib.dump(model0,"./model_scv_linear.joblib")

In [20]:
import joblib
from joblib import load
loaded_model0=joblib.load("model_scv_linear.joblib")

C:\Users\ylepen\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [21]:
Y_pred = loaded_model0.predict(X_test_tf)

In [22]:
Y_pred

array(['P3', 'P3', 'P3', ..., 'P3', 'P3', 'P3'], dtype=object)

In [ ]:
result = pd.DataFrame({'text': X_test.values,'actual':Y_test.values,'Predicted':Y_pred})

In [ ]:
result[result['actual']!=result['Predicted']].sample(10)

In [ ]:
class_names = ['P1','P2','P3','P4','P5']

In [ ]:
prob_Y = model0.predict_proba(X_test_tf)

In [ ]:
df=pd.DataFrame(prob_Y, columns=class_names)
df['P1']

In [ ]:
er=result.copy()
er.head()

In [ ]:
# new dataframe for explanation results
er = pd.DataFrame.join(er,df)


In [ ]:
er.sample(10, random_state = 10)

In [ ]:
id = 4016
print('document id: %d' %  id)


In [ ]:
print('Predicted priority = %s'% er.iloc[id]['Predicted'])
print('True Class  = %s' % er.iloc[id]['actual'])

In [ ]:
from lime.lime_text import LimeTextExplainer

In [ ]:
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
exp = explainer.explain_instance(result.iloc[id]['text'],pipeline.predict_proba,num_features=10,labels=[1,2])

In [ ]:
print('Explanation for class %s' % class_names[0])
print('\n'.join(map(str, exp.as_list(label=0))))
print()
print('Explanation for class %s' % class_names[2])
print('\n'.join(map(str, exp.as_list(label=2))))
print()

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
exp = explainer.explain_instance(result.iloc[id]['text'],pipeline.predict_proba,num_features=10,top_labels=5)
exp.show_in_notebook(text=False)

In [ ]:
from lime import submodular_pick
np.random.seed(12)
lsm=submodular_pick.SubmodularPick(explainer,er['text'].values, pipeline.predict_proba, sample_size=100,num_features=20,num_exps_desired=5)

In [ ]:
lsm.explanations[2].show_in_notebook()